In [14]:
import requests
import pprint
import geohash2

keyword = "양덕동"

# 최초 검색어에 해당하는 검색어값의 자동완성 ajax 주소 입니다.
# 예를 들어 사이트에서 대치동을 입력하면 대치동, 르엘대치(아파트), 대치동더블유타워(오피스텔)... 등의 
# 검색 결과목록이 나오는데 이 값을 구해오는 주소 입니다.
url = "https://apis.zigbang.com/search?q={}".format(keyword)

req = requests.get(url)

# 실제 api 주소에서 json 형태로 리턴되기 때문에 json 형태로 값을 받습니다.
# json 형태로 받은 값은 사용하기도 편리합니다.
_json = req.json()

# api 상태코드가 200인 경우가 오류없이 동작되었다는 의미입니다.
if _json.get("code") == "200":
    # 위에서 말한대로 검색어에 해당하는 자동완성값은 여러개인데 
    # 그중에 맨 위에 [0] 번째 한가지에 대해서만 검색을 합니다.
    data = _json.get("items")[0]
    _description = data.get("description")
    _id = data.get("id")
    _lat = data.get("lat")
    _lng = data.get("lng")
    _zoom = data.get("zoom")

    # 기존코드와 현재 변경된 직방 페이지에서 가장 중요하게 변경된 점은
    # 기존에는 lat, lng 값을 구해서 임의로 적정 영역을 +, - 해서 
    # 지도의 사각형 영역을 구한다음에 그 영역에 대한 쿼리를 요청했었는데
    # 변경된 직방 사이트는 Geohash 를 사용하도록 변경되었습니다.
    # Geohash 에 대한 정보는 https://en.wikipedia.org/wiki/Geohash 를 참고하시기 바랍니다.
    # 맨위에 설명한데로 파이썬 geohash2 라이브러를 먼저 설치해야 합니다.
    # precision 정밀도를 5로 설정해야만 직방에서 사용하는 geohash 와 일치하는듯 보입니다.
    geohash = geohash2.encode(_lat, _lng, precision=5)

    # 위에서 구한 geohash 값을 아래의 api 로 호출하고 쿼리(전세 월세 등)를 넘겨주는 주소 입니다.
    url = "https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세%7C월세&service_type_eq=원룸".format(geohash)

    # 역시 json 형태로 값을 취합니다.
    _req_items = requests.get(url).json()

    # json 데이터에서 items 값만 저장합니다.
    # items 값은 실제 매물 데이터의 인덱스 값입니다.
    _items = _req_items.get("items")

    # 위에서 취한 json 형태의 items 목록을
    # 파이썬 리스트 형태로 저장합니다.
    item_ids = []
    item_lat = []
    for item in _items:
        item_ids.append(item.get("item_id"))
        item_lat.append(item.get("lat"))
    # print(item_lat)

    # 위에서 저장한 list 의 100개만
    # items_ids 라는 키의 값으로 설정합니다.
    # 최종적으로 이 값을 직방 api 에 요청합니다.
    items = {"item_ids": item_ids[:2]}


    # 위에서 만든 items_ids: [매물인덱스] 를 아래 주소로 쿼리 한 후 json 형태로 받습니다.
    _results = requests.post('https://apis.zigbang.com/v2/items/list', data=items).json()

    # 최종 완성된 매물 결과는 items 안에 있습니다.
    datas = _results.get("items")

    print(datas)
    print("@@@@@@@@@@@@@@@")
    # 매물 목록을 돌며 화면에 출력합니다.
    for d in datas:
        _address = "{} {}".format(d.get("address1"), d.get("address2"))
        if d.get("address3") is not None:
            _address += " {}".format(d.get("address3"))

        building_floor = d.get("building_floor")
        floor = d.get("floor")
        thumbnail = d.get("images_thumbnail")
        item_id = d.get("item_id")
        reg_date = d.get("reg_date")
        sales_type = d.get("sales_type")
        service_type = d.get("service_type")
        size_m2 = d.get("size_m2")
        title = d.get("title")
        deposit = d.get("deposit")
        rent = d.get("rent")

        print("*" * 100)
        print("{} [{}]".format(title, item_id))
        print("보증금/월세: {}/{}".format(deposit, rent))
        print("건물층/매물층: {}/{}".format(building_floor, floor))
        print("등록일자: {}".format(reg_date))
        print("서비스형태/매물형태: {}/{}".format(service_type, sales_type))
        print("사이즈: {}".format(size_m2))

        print("****")
        atas = _results.get("items")
        print(atas)
        for a in atas:
            id = a.get("item_id")
            uurl = "https://apis.zigbang.com/v2/items/{}".format(id)
            specific = {}
            specific = requests.get(uurl).json()
            print("@@@@@@@@@@@@@@@@@@@@@")
            print(specific)
            
            print('*******************')
            print(specific["item"]["jibunAddress"])
            #print('specific' + specific.get("register").json())
            #for e in specific:
                #location = e.get("jibunAddress")
                #location = e
                #print(location)
                #print('structure' + str(location))


[{'section_type': None, 'item_id': 30263043, 'images_thumbnail': 'https://ic.zigbang.com/ic/items/30263043/1.jpg', 'sales_type': '월세', 'sales_title': '월세', 'deposit': 100, 'rent': 20, 'size_m2': 33.06, '공급면적': {'m2': 33.06, 'p': '10'}, '전용면적': {'m2': 33.06, 'p': '10'}, '계약면적': None, 'room_type_title': None, 'floor': '2', 'floor_string': '2', 'building_floor': '4', 'title': '🔵리모델링된 깔끔한 분리형원룸🔵', 'is_first_movein': None, 'room_type': '02', 'address': '포항시 북구 장성동', 'random_location': {'lat': 36.07939314174259, 'lng': 129.38490362067355}, 'is_zzim': False, 'status': True, 'service_type': '원룸', 'tags': [], 'address1': '경상북도 포항시 북구 장성동', 'address2': None, 'address3': None, 'manage_cost': '5', 'reg_date': '2022-01-17T15:27:29+09:00', 'is_new': False}, {'section_type': None, 'item_id': 30038206, 'images_thumbnail': 'https://ic.zigbang.com/ic/items/30038206/1.jpg', 'sales_type': '월세', 'sales_title': '월세', 'deposit': 200, 'rent': 25, 'size_m2': 26.45, '공급면적': {'m2': 26.45, 'p': '8'}, '전용면적': {'m2

@@@@@@@@@@@@@@@@@@@@@
{'item': {'item_id': 30038206, 'user_no': 433235, 'sales_type': '월세', 'sales_title': '월세', 'service_type': '원룸', 'images': ['https://ic.zigbang.com/ic/items/30038206/1.jpg', 'https://ic.zigbang.com/ic/items/30038206/2.jpg', 'https://ic.zigbang.com/ic/items/30038206/3.jpg', 'https://ic.zigbang.com/ic/items/30038206/4.jpg', 'https://ic.zigbang.com/ic/items/30038206/5.jpg', 'https://ic.zigbang.com/ic/items/30038206/6.jpg', 'https://ic.zigbang.com/ic/items/30038206/7.jpg', 'https://ic.zigbang.com/ic/items/30038206/8.jpg'], 'image_thumbnail': 'https://ic.zigbang.com/ic/items/30038206/1.jpg', '매매금액': None, '보증금액': 200, '월세금액': 25, '전용면적_m2': 26.45, '공급면적_m2': 26.45, '대지권면적_m2': None, 'address': '포항시 북구 양덕동', 'jibunAddress': '포항시 북구 양덕동 1668', 'local1': '경상북도', 'local2': '포항시 북구', 'local3': '양덕동', 'local4': None, 'room_type_code': '02', 'room_type': '02', 'title': '⭐⭐완전풀옵션 넓고 깔끔한방⭐⭐', 'status': 'open', 'description': '🖤양덕동 하나로마트 건너편에 위치해 있습니다\r\n🖤타원룸에 비해 방이 크게 빠져 넓게 쓰시기 